# Combinando Dataframes
Existen diferentes formas de fusionar dos dataframes. Esto se hace a través de la lógica de combinación:

* **left join**: Da prioridad al Dataframe de la izquierda. Trae siempre los datos de la izquierda y las filas en común que tenga con el Dataframe de la derecha

* **rigth join**: Da prioridad al Dataframde d la derecha. Trae siempre los datos de la derecha y las filas en común con el dataframe de la izquierda

* **inner join**: Trae solamente aquellos datos que son común en ambos dataframe's, es decir los datos que hagan "match"

* **outer join**: Trae los datos tanto del dataframe de la izquierda, como de la derecha, incluyendo los datos que comparten ambos.


   

## hacemos cosas aburridas

Dado que vamos a hacer los df's desde listas $\rightarrow$  listas de listas $\rightarrow$ dicts $\rightarrow$ df's, dejaremos el código que crea los diccionarios a partir de listas en un pyfile. El cuál estaremos importando desde su ubicación, hacia este nb

In [1]:
import numpy as np
import pandas as pd
import os
import sys

In [2]:
CURRENT_DIR = os.getcwd()
SCRIPT_DIR = os.path.join(CURRENT_DIR, os.pardir, "utils")

In [3]:
#[os.path.join(SCRIPT_DIR, item) for item in os.listdir(SCRIPT_DIR)]

In [4]:
sys.path.insert(0 , SCRIPT_DIR)

In [5]:
import utils

In [6]:
dict_a1, dict_a2, dict_b1, dict_b2, dict_c2, dict_d1, dict_d2 = utils.data_json()

## Método .concat( )
Cuando el match es por medio de filas
* teniendo dos datasets y se requiere fusionarlos a nivel de filas  usamos `axis = 0`
    * se hace un "stack" (se apilan) entre los dataframes, crece verticalmente

Cuando el match es por columnas
* teniendo dos datasets que se requieren fusionar por columnas usamos `axis = 1`
  * habrán valores núlos porque la organización por columnas no resperará la estructura que tienen

  


In [7]:
# trabajaremos con los siguientes json
print(f' df1: \n {dict_a1}\n\n df2: \n{dict_a2}')


 df1: 
 {'A': ['A0', 'A1', 'A2', 'A3'], 'B': ['B0', 'B1', 'B2', 'B3'], 'C': ['C0', 'C1', 'C2', 'C3'], 'D': ['D0', 'D1', 'D2', 'D3']}

 df2: 
{'A': ['A4', 'A5', 'A6', 'A7'], 'B': ['B4', 'B5', 'B6', 'B7'], 'C': ['C4', 'C5', 'C6', 'C7'], 'D': ['D4', 'D5', 'D6', 'D7']}


In [8]:
#Pasamos a df los dicts
dicts_a = [dict_a1, dict_a2]
df_a1, df_a2 = [pd.DataFrame(i) for i in dicts_a]

### .concat( ) y .merge( )

Para concatenar dos ó más df's usamos la función `df.concat()` esto stackea las filas del df anterior, teniendo como argumento:
  * lista de `[df's]`
  * `ingore_index = True` corregir los índices a la hora de **stackear las filas*
  * `axis =` especificar si se concatena por columnas=1 ó por filas=0

Esto no hace más que agregar los elementos a las columnas que tienen en común (left join) en caso de que durante el proceso de concatenación, haya elementos donde en la columna no tengan valores en esa columna, ese espacio será núlo 
 * Al concatenar, se colocan también los índices el df agregado, para que los índices sean 

así : `pd.concat([dict_1, dict_2], ignore_index= True, axis = 0)`

In [9]:
# por columnas
df_concat_cols = pd.concat([df_a1,df_a2], axis = 1)

#por filas
df_concat_rows = pd.concat([df_a1, df_a2], ignore_index = True, axis=0 )
df_concat_rows

,A,B,C,D
0,A0,B0,C0,D0
1,A1,B1,C1,D1
2,A2,B2,C2,D2
3,A3,B3,C3,D3
4,A4,B4,C4,D4
5,A5,B5,C5,D5
6,A6,B6,C6,D6
7,A7,B7,C7,D7


### merge( ) : label match
`df_izq.merge(df_der)`

Un df estará invocando a la función `merge()`y el otro estará como argumento de la función 
* Para especificar que va a ser un *merge* sobre una columna en específico se usa el parámetro `on = 'columna'`

### sin columnas en común 
Cuando los df's no tienen columnas en común, los df's no se fusionarán. hay que aclarar las columnas con las que se hará el merge con los siguientes parámetros:
* `right_on = 'columna'` columna del df de la izquierda
* `left_on = 'columna'` columnas del df de la derecha

*dados dos df's con columnas distintas*

así : `df_izq.merge(df_der, left_on= 'col_izq', rigth_on = 'col_der')`

### valores núlos en el df
 Si tenemos un Nan en el df, pandas no lo detectará como un "match", se soluciona agregando un parámetro más a la fusión: `how = `, que establece el tipo de fusión, siendo:
  * `left`  usa los elementos del df izquierdo
  * `right` usa los elementos del df derecho
  * `outer` usa TODOS los elementos que están en ambos df's, ordenados con el df que invoca a la función(df de la izquierda) con el otro
  * `inner` usa solo los elementos que están en ambos df's (los que tienen en común)
  * `cross` usa el *producto cartesiano* entre ambos df's perservando el orden de los elementos de la izquierda
     * sería en relación entre cada una de las filas de los df's con los del otro: como si fueran coordenadas, y tratando al de la izquierda como "x"

así: `df_izq.merge(df_der, left_on= 'col_izq', right_on = 'col_der', how = 'metodo_fusión')`

In [10]:
dicts_b = [dict_b1, dict_b2]
izq, der = [pd.DataFrame(i) for i in dicts_b]
izq

,key,A,B
0,k0,A0,B0
1,k1,A1,B1
2,k2,A2,B2
3,k3,A3,B3


primer merge: outer join

El siguiente es un merge de columnas df_izquierda, con las que tienen columnas en común y con las que no con df_derecha

In [22]:
# merge sin especificar por qué columnas se usarán para la fusión 
# izq.merge(der)

# merge sobre una columna específica (key) que está en ambas columnas
izq.merge(der, on = 'key')

,key,A,B,C,D
0,k0,A0,B0,C0,D0
1,k1,A1,B1,C1,D1
2,k2,A2,B2,C2,D2
3,k3,A3,B3,C3,D3


Segundo merge, sin columnas en común: 

In [12]:
dicts_c = [dict_b1, dict_c2]
izq_2, der_2 = [pd.DataFrame(i) for i in dicts_c]


In [13]:
#merge con columnas diferentes, indicando qué columnas izq/der se hará 
izq_2.merge(der_2,  left_on= 'key', right_on = 'key_2')

,key,A,B,key_2,C,D
0,k0,A0,B0,k0,C0,D0
1,k1,A1,B1,k1,C1,D1
2,k2,A2,B2,k2,C2,D2
3,k3,A3,B3,k3,C3,D3


In [29]:

pd.concat([izq_2 , der_2 ], axis = 1)

,key,A,B,key_2,C,D
0,k0,A0,B0,k0,C0,D0
1,k1,A1,B1,k1,C1,D1
2,k2,NaN,B2,k2,C2,D2
3,k3,A3,B3,k3,C3,D3


Tercer merge: con calores núlos

In [36]:
dict_null = {'key': ['k0', 'k1', 'k2', np.nan],
 'A': ['A0', 'A1', 'A2', 'A3'],
 'B': ['B0', 'B1', 'B2', 'B3']}

dicts_c = [dict_null, dict_c2]
izq_2, der_2 = [pd.DataFrame(i) for i in dicts_c]
izq_2.merge(der_2,  left_on= 'key', right_on = 'key_2')

,key,A,B,key_2,C,D
0,k0,A0,B0,k0,C0,D0
1,k1,A1,B1,k1,C1,D1
2,k2,A2,B2,k2,C2,D2


Dado que los elementos de las columnas de intersección son diferentes (en un df hay un núlo y en el otro no) al hacer un *inner* join se descarta la fila donde no hay match entre los dos df's

* para que salgan todas las columnas habiendo hecho match ó no usamos `how = 'left'` para traer esa  fila con el núlo

In [38]:
izq_2.merge(der_2, left_on = 'key', right_on = 'key_2', how = 'left')

,key,A,B,key_2,C,D
0,k0,A0,B0,k0,C0,D0
1,k1,A1,B1,k1,C1,D1
2,k2,A2,B2,k2,C2,D2
3,NaN,A3,B3,NaN,NaN,NaN


Traemos a los matches de las columnas, junto con todas las filas del df de la izquierda.
 * en de la derecha  la fila 3  la trae, pero con valores nan. Los elementos del df de la derecha sí existen en la fila 3 ¿Por qué son núlos?
   * porque se está haciendo match teniendo en cuenta los *key / key_2* de los df's. Dado que estos *keys* del df izquierda/derecha no son iguales en una fila, no hace macth.
     *  Entonces no se trae esa fila y por lo tanto no se toma en cuenta.

### join( ) : index match
Casi igual que `merge`. La diferencia es que join va a los **índices** y no a las columnas específicas
* `join` busca el match a través de índices y no de columnas específicas

Para traer todos los datos, aunque no hagan match usamos el parámetro `how = outer`
 * recordar que *outer* trae todos los valores de ambos df's

así: `df_izq.join(df_der, how = 'outer')`

Cuando sale un *NaN* en filas puede ser por dos razones: 
 * porque el valor en cuestión es núlo
 * porque a pesar de haber valores en esa posición, no hay match con la columna principal de comparación

 Respecto a merge y join: Dado que join trabaja con los índices. Merge da un mejor control en cuanto a poder elegir con cuál columna queremos hacer la fusión en ambos lugares. 


In [40]:
izq_3 = pd.DataFrame(dict_d1, index = ['k0', 'k1', 'k2', 'k3'])
der_3 = pd.DataFrame(dict_d2, index = ['k0', 'k2', 'k4', 'k5'])
print(izq_3, "\n\n", der_3)

     A   B
k0  A0  B0
k1  A1  B1
k2  A2  B2
k3  A3  B3 

      C   D
k0  C0  D0
k2  C1  D1
k4  C2  D2
k5  C3  D3


In [47]:
# traer todos los elementos de ambos df's sin imporatar intersección
izq_3.join(der_3, how = 'outer')

,A,B,C,D
k0,A0,B0,C0,D0
k1,A1,B1,NaN,NaN
k2,A2,B2,C1,D1
k3,A3,B3,NaN,NaN
k4,NaN,NaN,C2,D2
k5,NaN,NaN,C3,D3


In [43]:
# traer los elementos de intersección entre ambos 
izq_3.join(der_3, how = 'inner') 

,A,B,C,D
k0,A0,B0,C0,D0
k2,A2,B2,C1,D1


In [44]:
# traer todos los de la izquierda y los elementos de intersección entre ambos
izq_3.join(der_3, how = 'left')

,A,B,C,D
k0,A0,B0,C0,D0
k1,A1,B1,NaN,NaN
k2,A2,B2,C1,D1
k3,A3,B3,NaN,NaN


In [45]:
#traer todos los elementos de la derecha y los de intersección entre ambos
izq_3.join(der_3, how = 'right')

,A,B,C,D
k0,A0,B0,C0,D0
k2,A2,B2,C1,D1
k4,NaN,NaN,C2,D2
k5,NaN,NaN,C3,D3


In [46]:
# traer la relación de producto cartesiano entre ambos 
# siendo así -> filas:(0,0)->(0,1)->(0,2)->(0,3)->(1,0)->(1,1)->(1,2) sucesivamente
izq_3.join(der_3, how = 'cross')

,A,B,C,D
0,A0,B0,C0,D0
1,A0,B0,C1,D1
2,A0,B0,C2,D2
3,A0,B0,C3,D3
4,A1,B1,C0,D0
5,A1,B1,C1,D1
6,A1,B1,C2,D2
7,A1,B1,C3,D3
8,A2,B2,C0,D0
9,A2,B2,C1,D1
